In [2]:
from pyspark.sql import functions as sf, DataFrame, SparkSession

spark = SparkSession.builder.getOrCreate()

from feature_logic.main import create_features
from feature_logic.feature_groups.test_group import LocationFeatures

22/10/21 14:10:30 WARN Utils: Your hostname, Roels-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 172.16.232.237 instead (on interface en0)
22/10/21 14:10:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/21 14:10:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data = spark.createDataFrame([dict(party_id="a"), dict(party_id="b"), dict(party_id="c")])

location_data = spark.createDataFrame([
    dict(party_id="a", postcode="1234AB"),
    dict(party_id="b", postcode="1234BC"),
    dict(party_id="c", postcode=""),
])
try:
    location_data.write.parquet("data/location_data.parquet")
except:
    pass

In [5]:
features = create_features(
    data=data,
    level="party", 
    feature_groups=[
        LocationFeatures(spark=spark, features=["postcode"]),
    ])

['party_id'] ['party_id']
<feature_logic.feature_groups.test_group.LocationFeatures object at 0x1072ea9b0> apply ['party_id']
[Column<'location.postcode AS postcode'>]


In [6]:
features.toPandas()

,party_id,postcode
0,a,1234AB
1,b,1234BC
2,c,


In [7]:
stop

NameError: name 'stop' is not defined

In [ ]:

wow = spark.read.parquet("wow.parquet")
wow.limit(10).toPandas()

In [ ]:
wow.select("zone").distinct().collect()

In [ ]:
(
    wow
    .withColumn("durotar_flag", sf.when(sf.col("zone") == " Durotar", 1).otherwise(0))
    .groupby("avatarId")
    .agg(
        sf.count("*").alias("total_count"),
        sf.sum("durotar_flag").alias("durotar_count")
    )
    .withColumn("durotar_prob", sf.col("durotar_count") / sf.col("total_count"))
    .show()
)